In [ ]:
%load_ext dotenv
%dotenv

import os
from sodapy import Socrata

In [ ]:
# Print URL

import requests
import logging
import http.client

http.client.HTTPConnection.debuglevel = 1

logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)
requests_log = logging.getLogger("requests.packages.urllib3")
requests_log.setLevel(logging.DEBUG)
requests_log.propagate = True

In [ ]:
client = Socrata(
    os.environ["SOCRATA_DOMAIN"],
    os.environ["SOCRATA_APP_TOKEN"],
    username=os.environ["SOCRATA_API_KEY_ID"],
    password=os.environ["SOCRATA_API_KEY_SECRET"],
    timeout=60,
)

In [ ]:
metadata = client.get_metadata(os.environ["DATASET_ID"])
[(col["name"], col["fieldName"], col["dataTypeName"]) for col in metadata["columns"]]

In [ ]:
dataset = client.get(
    os.environ["DATASET_ID"],
    select="town, listyear, saleamount, salesratio",
)
dataset

In [ ]:
query = """
select count(*)
"""
resp = client.get(
    os.environ["DATASET_ID"],
    query=query,
)
total_items = int(resp[0]["count"])
total_items

In [ ]:
query = """
select distinct listyear
"""
resp = client.get(
    os.environ["DATASET_ID"],
    query=query,
)
years = sorted([item["listyear"] for item in resp])
years

In [ ]:
# Sales volume
# ============
year = years[-1]
query = f"""
select town, count(serialnumber), sum(saleamount) as salesvolume
where listyear = {year}
group by town
order by salesvolume desc
limit 10
"""
resp = client.get(
    os.environ["DATASET_ID"],
    query=query,
)
resp

In [ ]:
# Sales ratio SoQL
# ================
year = years[-1]
query = f"""
select town, avg(cast(salesratio as number)) as salesratioavg
where listyear = {year}
order by salesratio desc
limit 10
"""
resp = client.get(
    os.environ["DATASET_ID"],
    query=query,
)
resp

In [ ]:
# Sales ratio pandas
# ==================
import pandas as pd
import numpy as np

year = years[-1]
query = f"""
select count(*)
where listyear = {year}
"""
resp = client.get(
    os.environ["DATASET_ID"],
    query=query,
)
total_items = int(resp[0]["count"])
page_size = 1000
pages = total_items // page_size + 1 if total_items % page_size else 0

rows = []

for page_number in range(pages):
    query = f"""
    select town, salesratio
    where listyear = {year}
    order by serialnumber
    offset {page_number * page_size}
    limit {page_size}
    """
    resp = client.get(
        os.environ["DATASET_ID"],
        query=query,
    )
    rows.extend(resp)
df = pd.DataFrame.from_records(rows)
df["salesratio"] = pd.to_numeric(df.salesratio)
df.groupby("town").agg(np.average).sort_values(by="salesratio", ascending=False)[:10]


In [ ]:
df2 = df.groupby("town").agg(np.average).sort_values(by="salesratio", ascending=False)[:10]
df2.reset_index().values.tolist()